# Import Library & Load Data

In [1]:
import pandas as pd
import numpy as np
import emoji
import re
from collections import Counter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import tqdm
from tqdm.auto import tqdm 
tqdm.pandas()

# load raw data
df = pd.read_csv('../data/raw/shopee_reviews_raw.csv')

# check sample data before cleaning
print("Sample Raw Data:")
print(df['review_text'].head(10))

Sample Raw Data:
0    padahal aku nyamaaan banget belaja disini tapi...
1                                         bagus bangat
2    semua oke sih, cuma saat saya mau ngasih ulasa...
3                                   mudah simpel cepat
4    maaf untuk sementara saya kasih bintang 1, kar...
5                                                bagus
6    aplikasi aneh, transaksi selalu dibatalkan sys...
7                                               mantap
8    sistem sangat bobrok, berulangkali mendapat or...
9    Akun tiba tiba diblokir tidak ada penjelasan, ...
Name: review_text, dtype: object


# Pre-Cleaning EDA
Data Profiling Analysis:
- Vocabulary Size Check: Counts the number of unique words in the raw data.
- Top Noise Identification: Looks for frequently occurring words that are not standard. This is a resource for scientifically creating key_norms (slang dictionaries).
- Emoji Analysis: Sees which emojis appear most frequently. The 😊 or 😡 emojis are important features for sentiment analysis.

In [2]:
# vocabulary size check
raw_all_words = df['review_text'].str.lower().str.split().explode().dropna()
raw_vocab_size = raw_all_words.nunique()

print(f"Number of unique words (Vocabulary) before cleaning: {raw_vocab_size}")

Number of unique words (Vocabulary) before cleaning: 20985


In [3]:
# check 20 most frequent words (including slang/noise)
top_tokens = raw_all_words.value_counts().head(20).rename_axis('Word').reset_index(name='Frequency')

print("\n20 most frequent words (check for potential stopwords/slang):")
print(top_tokens)


20 most frequent words (check for potential stopwords/slang):
          Word  Frequency
0           di       4759
1       shopee       3995
2       sangat       3441
3          dan       3377
4         saya       2615
5        bagus       2598
6      belanja       2280
7          nya       1997
8       barang       1877
9           yg       1852
10        yang       1706
11         ada       1598
12       tidak       1595
13        bisa       1573
14    aplikasi       1510
15      mantap       1289
16    membantu       1254
17  pengiriman       1253
18         ini       1216
19       untuk       1059


In [4]:
# 3. check most frequent emojis
import emoji

all_emojis = df['review_text'].apply(lambda t: [c for c in t if c in emoji.EMOJI_DATA]).explode().dropna()
emoji_counts = all_emojis.value_counts().head(20).rename_axis('Emoji').reset_index(name='Frequency')

print('\Most frequent emojis:')
print(emoji_counts)

\Most frequent emojis:
   Emoji  Frequency
0      👍       1561
1      ⭐        597
2      🙏        336
3      🏻        244
4      🥰        217
5      😡        215
6      👎        121
7      😭        110
8      ❤         91
9      😍         90
10     👌         79
11     😊         69
12     😁         59
13     💯         54
14     😘         42
15     🫰         41
16     🌟         38
17     🤩         36
18     😄         36
19     🤗         35


# Normalization maps

In [5]:
emoji_map = {
'👍': ' bagus ', '💯': ' bagus ', '😡': ' marah ', '😠': ' marah ',
'⭐': ' bintang ', '🌟': ' bintang ', '❤️': ' suka ', '❤': ' suka ',
'🥰': ' suka ', '😍': ' suka ', '🤩': ' suka ', '😘': ' suka ',
'🫰': ' suka ', '👌': ' oke ', '😊': ' senang ', '😁': ' senang ',
'🤣': ' lucu ', '🙏': ' terima kasih ', '😔': ' kecewa ', '👎': ' buruk '
}


key_norm = {
'yg': 'yang', 'gan': 'juragan', 'n': 'dan', 'bgt': 'banget', 'nga': 'tidak',
'gk': 'tidak', 'gak': 'tidak', 'ga': 'tidak', 'nggak': 'tidak', 'shoppee': 'shopee',
'sdh': 'sudah', 'udah': 'sudah', 'blm': 'belum', 'dgn': 'dengan',
'tlg': 'tolong', 'brg': 'barang', 'sy': 'saya', 'kalo': 'kalau',
'klo': 'kalau', 'jm': 'jam', 'bs': 'bisa', 'krn': 'karena', 'lelet': 'lambat',
'tp': 'tapi', 'dr': 'dari', 'kpn': 'kapan', 'jg': 'juga', 'gede': 'besar',
'ok': 'oke', 'thx': 'terima kasih', 'plis': 'tolong', 'bener': 'benar',
'good': 'bagus', 'bagu': 'bagus', 'mantap': 'bagus', 'cepet': 'cepat',
'jelek': 'buruk', 'lemot': 'lambat', 'seneng': 'senang'
}

In [6]:
def clean_text_regex(text):
    if not isinstance(text, str):
        return ''
    
    # 1. lowercase
    text = text.lower()
    
    # 2. replace Emojis using emoji_map
    for emo, replacement in emoji_map.items():
        text = text.replace(emo, replacement)
    text = emoji.replace_emoji(text, replace='')
    
    # 4. delete URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # 5. delete mention (@) and hashtag (#)
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # 6. delete any special characters and numbers (only keep letters a-z and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 7. delete excessive repeated characters (ex: "baguuusss" -> "bagus")
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    
    # 8. delete excessive spaces (multiple spaces -> single space)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
def normalize_slang_fast(text):
    """Mengganti slang menggunakan Dictionary Lookup (Sangat Cepat)."""
    if not isinstance(text, str):
        return ''
    
    # split sentence to words
    words = text.split()
    
    # list comprehension: check dictionary, if exists replace, else keep
    normalized_words = [key_norm.get(word, word) for word in words]
    
    return ' '.join(normalized_words)

# Setup Stopwords & Stemmer (Sastrawi)
For sentiment analysis, it's best not to remove negative words like "tidak" "bukan," or "jangan." Sastrawi removes "tidak" by default, so it needs to be modified.

In [8]:
# setup stopwords
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

# exclude negative words from stopwords
excluded_stopwords = ['tidak', 'bukan', 'jangan', 'belum', 'tapi']
stopwords = [word for word in stopwords if word not in excluded_stopwords]

# setup stemmer
stemmer = StemmerFactory().create_stemmer()

def remove_stopwords(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word in text.split() if word not in stopwords])

def stemming_text(text):
    if not isinstance(text, str):
        return ''
    return stemmer.stem(text)

# Pipeline Execution (Batch Processing)

In [9]:
print("Starting cleaning process... ")

# apply clean text (normalize & regex)
df['text_clean'] = df['review_text'].progress_apply(clean_text_regex)
df['text_clean'] = df['text_clean'].progress_apply(normalize_slang_fast)

# apply stopword removal
df['text_clean'] = df['text_clean'].progress_apply(remove_stopwords)

# apply stemming
df['text_clean'] = df['text_clean'].progress_apply(stemming_text) 

print("Cleaning process completed!")

# compare before vs after
print("\nData Comparison:")
print(df[['review_text', 'text_clean']].head(10))

Starting cleaning process... 


  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

Cleaning process completed!

Data Comparison:
                                         review_text  \
0  padahal aku nyamaaan banget belaja disini tapi...   
1                                       bagus bangat   
2  semua oke sih, cuma saat saya mau ngasih ulasa...   
3                                 mudah simpel cepat   
4  maaf untuk sementara saya kasih bintang 1, kar...   
5                                              bagus   
6  aplikasi aneh, transaksi selalu dibatalkan sys...   
7                                             mantap   
8  sistem sangat bobrok, berulangkali mendapat or...   
9  Akun tiba tiba diblokir tidak ada penjelasan, ...   

                                          text_clean  
0  padahal aku nyaman banget baja sini tapi opsi ...  
1                                       bagus bangat  
2  semua oke sih cuma mau ngasih ulas unggah vide...  
3                                 mudah simpel cepat  
4  maaf kasih bintang beku akun lama banget buk p...  
5      

# Measuring Success

In [10]:
# count unique words in cleaned text
clean_vocab = set()
for text in df['text_clean']:
    for word in text.split():
        clean_vocab.add(word)

print(f"First vocabulary: {len(raw_all_words)}")
print(f"Final vocabulary: {len(clean_vocab)}")
print(f"Noise Reduction Efficiency: {((len(raw_all_words) - len(clean_vocab)) / len(raw_all_words)) * 100:.2f}%")

# display random 5 samples of before vs after cleaning
comparison = df[['review_text', 'text_clean']].sample(5)
display(comparison)

First vocabulary: 187647
Final vocabulary: 11020
Noise Reduction Efficiency: 94.13%


,review_text,text_clean
2789,makin gacor..wkwk,makin gacorwkwk
18050,sumpah Shopee bagus tp² kenapa tiba² saya disu...,sumpah shopee bagus tapi tiba suruh bayar mulu...
7050,cukup puas untuk kedepan nya semoga shopee leb...,cukup puas depan nya moga shopee lebih baik mu...
9379,banyak penipuan,banyak tipu
11883,sangat membantu.blanja jdi mudah dari berbagai...,sangat membantublanja jdi mudah bagai kalang


# Save Clean Data

In [11]:
# save clean data
output_path = '../data/processed/shopee_reviews_clean.csv'
df.to_csv(output_path, index=False)
print(f"Clean data saved at: {output_path}")

Clean data saved at: ../data/processed/shopee_reviews_clean.csv
